In [2]:
%run utils/TransformCoordinates.ipynb

import re
import colorama
from colorama import Fore

class QualityControl(TransformCoordinates):
    
    def __init__(self, fiducial, component, trayPosition = 'P1'):
        TransformCoordinates.__init__(self, fiducial, component, trayPosition)
        self.trayPosition = trayPosition
        self.degrees = [0.5, 1.0, 2.0, 90.]
        self.classify = {'-1': Fore.MAGENTA + 'NO INFO' + Fore.BLACK, 
                       '0': Fore.GREEN + 'GREEN' + Fore.BLACK,
                       '1': Fore.YELLOW + 'YELLOW' + Fore.BLACK, 
                       '2': Fore.BLUE + 'BLUE' + Fore.BLACK, 
                       '3': Fore.RED + 'RED' + Fore.BLACK}
        self.centers = [0.025, 0.050, 0.100, 10.0]
        self.h2 = [0.025, 0.050, 0.100, 10.0]
        self.h1 = [-1., 0.025, 0.050, 0.100]
        self.nominalHeights = {'CuW': 3.100, 'PCB': 3.100}
        self.QC = [-1]
        self.finalRating = ''
        self.heightNotes = ''
        self.angleNotes = ''
        self.placementNotes = ''
        self.finalNotes = ''
        self.notes = ''
        
    def __str__(self):
        output = Fore.BLACK + 'The module/component in ' + self.trayPosition + ' has completed QC!\n'
        output += 'Rotation --> Left Fiducial: {0:.3f} degrees, Right Fiducial: {1:.3f} degrees\n'.format(self.deg[0], self.deg[1])
        output += self.angleNotes
        output += 'Placement --> (x,y) = ({0:.3f}, {1:.3f}) mm\n'.format(self.center[0], self.center[1])
        output += self.placementNotes
        output += 'Height --> {0:.3f} + ({1:.3f}) - ({2:.3f}) mm\n'.format(self.hgt[0], self.hgt[1], self.hgt[2])
        output += self.heightNotes
        output += self.finalNotes
        
        return output
        
    #Checking the quality of the angle of the placed component with respect to the tray position
    def angleQuality(self):
        self.degQC = -1
        for i, d in enumerate(self.degrees):
            if (abs(float(self.deg[0] - 90.)) < d or abs(float(self.deg[1] - 90.)) < d):
                self.angleNotes = 'The angle of the sensor in position ' + self.trayPosition + ' is ' + self.classify[str(i)] + '. \n'
                self.degQC = i
                break
        self.notes += Style.RESET_ALL + self.angleNotes
        self.QC.append(self.degQC)
        
    #Position quality checking for the component center
    def positionQuality(self):
        self.posQC = -1
        for i, p in enumerate(self.centers):
            if self.centerPosition < p:
                self.placementNotes = 'The placement of the sensor in position ' + self.trayPosition + ' is ' + self.classify[str(i)] + '. \n'
                self.posQC = i
                break
        self.notes += Style.RESET_ALL + self.placementNotes
        self.QC.append(self.posQC)
              
    #Height measurement means/max/min is within the allowed nominal value for each module type
    def heightQuality(self):
        self.hgtQC = -1
        for i, h in enumerate(self.h2):
            if abs(self.hgt[0] - self.nominalHeights['CuW']) < h and abs(self.hgt[0] - self.nominalHeights['CuW']) >= self.h1[i]:
                self.heightNotes += 'The mean heights of the module in position ' + self.trayPosition + ' are ' + self.classify[str(i)] + '. \n'
                self.hgtQC = max(self.hgtQC, i)
            if self.hgt[1] < h and self.hgt[1] >= self.h1[i]:
                self.heightNotes += 'The minimum height of the module in position ' + self.trayPosition + ' is ' + self.classify[str(i)] + '. \n'
                self.hgtQC = max(self.hgtQC, i)
            if self.hgt[2] < h and self.hgt[2] >= self.h1[i]:
                self.heightNotes += 'The maximum height of the module in position ' + self.trayPosition + ' is ' + self.classify[str(i)] + '. \n'
                self.hgtQC = max(self.hgtQC, i)
                
        self.notes += Style.RESET_ALL + self.heightNotes
        self.QC.append(self.hgtQC)
                
    #Final rating function that looks at each measurement type and classifies as GREEN/YELLOW/ORANGE/RED
    #We get a final rating from this also
    def rating(self):
        if len(self.hgt): self.heightQuality()
        if len(self.deg): self.angleQuality()
        if self.centerPosition >= 0.: self.positionQuality()
        rate = max(self.QC)
        self.finalRating = self.classify[str(rate)]
        
        self.finalNotes = '\nThe FINAL rating of the module in position ' + self.trayPosition + ' is ' + self.finalRating + '.\n'
        self.notes += Style.RESET_ALL + self.finalNotes

Exception: File `'utils/TransformCoordinates.ipynb.py'` not found.